In [125]:
# Scrapping from reclameaqui.com.br
import pandas as pd
import time
import random
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.firefox.service import Service
import os
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc
import setuptools._distutils


In [126]:
# nome da empresa seguindo convenção do site reclameaqui.com.br
empresa : str = 'coin-nacional'

# URL da pagina principal da empresa
url_p = 'https://www.reclameaqui.com.br/empresa/' + empresa + '/'

# URL da pagina de reclamações da empresa
def url_r(page : int, type : str = 'EVALUATED') -> str:

    # https://www.reclameaqui.com.br/empresa/temu/lista-reclamacoes/?pagina=1&status=EVALUATED
    return 'https://www.reclameaqui.com.br/empresa/' + empresa + '/lista-reclamacoes/?pagina='+ str(page) +'&status=' + type

# driver
driver = uc.Chrome()


In [128]:
# pegando metadados da empresa (nome, categoria, reputação, sobre)

if not os.path.exists(f'metadados/metadados_{empresa}.csv'):
   # acessando a pagina principal da empresa
   driver.get(url_p)

   time.sleep(random.randint(4,6))
      
   # ramo de atividade da empresa
   categoria = driver.find_element(By.ID, 'info_segmento_hero').text

   # reputação da empresa
   reputacao = driver.find_element(By.ID, 'tag_reputacao_hero').text

   # sobre a empresa
   sobre = driver.find_element(By.CSS_SELECTOR, 'ul.sc-1915fv4-2.idgfcC').text
   sobre = sobre.split("\n")[0] 

   metadata = {
      'nome': empresa,
      'categoria': categoria,         
      'reputacao': reputacao,         
      'sobre': sobre       
   }
   
   metadata_p = pd.DataFrame(metadata, index=[0])
   metadata_p.to_csv(f'metadados/metadados_{empresa}.csv', index=False)

else:
   
   metadata = pd.read_csv(f'metadados/metadados_{empresa}.csv')
   metadata = metadata.to_dict(orient='records')[0]
   
   
print(metadata)




{'nome': 'coin-nacional', 'categoria': 'Bancos e Financeiras', 'reputacao': 'Ótimo', 'sobre': 'Negociar dívidas nunca foi tão fácil. Segurança e eficiência ao seu lado.'}


In [129]:
# pegando hyperlinks das reclamações avaliadas e/ou respondidas

all_hyperlinks = []
Numero_de_paginas = 50  # máximo de 50 páginas

# Se o CSV ainda não existe, inicia a coleta
if not os.path.exists(f'hyperlinks/hyperlinks_{empresa}.csv'):
   
   for i in range(Numero_de_paginas):
      try:
         driver.get(url_r(page=i + 1, type='EVALUATED'))

      
         # Captura os hyperlinks da página atual
         div_hyperlinks = driver.find_elements(By.CSS_SELECTOR, 'div.sc-1pe7b5t-0.eJgBOc')
         current_hyperlinks = [h.find_element(By.CSS_SELECTOR, 'a').get_attribute('href') for h in div_hyperlinks]
         
         if not current_hyperlinks:
               print("Nenhum hyperlink encontrado")
               break

         print(f"Página {i+1} - hyperlinks: {current_hyperlinks}")
         all_hyperlinks += current_hyperlinks

         
         time.sleep(random.randint(5, 8))

        
      except Exception as e:
         print(f"Erro encontrado: {e}")
         break


   df = pd.DataFrame(all_hyperlinks, columns=["Hyperlinks"])
   df.to_csv(f'hyperlinks/hyperlinks_{empresa}.csv', index=False)
   all_hyperlinks = df["Hyperlinks"].tolist()
    
else:

   # Se o CSV já existe, apenas o lê
   df = pd.read_csv(f'hyperlinks/hyperlinks_{empresa}.csv')
   all_hyperlinks = df["Hyperlinks"].tolist()

print(all_hyperlinks)
   

['https://www.reclameaqui.com.br/coin-nacional/reclamacao_cFbivb8b6Qr847sT/', 'https://www.reclameaqui.com.br/coin-nacional/preciso-do-contato_yECcVrT4pGNSN9un/', 'https://www.reclameaqui.com.br/coin-nacional/reclamacao_PbVPtZ9tf2VLNlDX/', 'https://www.reclameaqui.com.br/coin-nacional/nao-consigo-contato_C1Ww7WcFR6VgFgFf/', 'https://www.reclameaqui.com.br/coin-nacional/boleto_l72PQ0w1pPwshMy5/', 'https://www.reclameaqui.com.br/coin-nacional/empresa-enganosa_XtrhjOaJxKNImJld/', 'https://www.reclameaqui.com.br/coin-nacional/valor-cobrado_9UQ-3VaBQ3ccJsHv/', 'https://www.reclameaqui.com.br/coin-nacional/aplicativo_7XNttevY2P6QA17c/', 'https://www.reclameaqui.com.br/coin-nacional/quitacao_Xm7QFIqsq5UrnsEH/', 'https://www.reclameaqui.com.br/coin-nacional/boleto_ng_EWwFuF12O-x-9/', 'https://www.reclameaqui.com.br/coin-nacional/cancelamento-de-contrato_OXO49y8coeHKknYA/', 'https://www.reclameaqui.com.br/coin-nacional/a-empresa-coin_3Pq1gxcjO1Pu1uIs/', 'https://www.reclameaqui.com.br/coin-naci

In [130]:

# pegando dados das reclamações respondidas



if not os.path.exists(f'reclamacoes/reclamacoes_{empresa}.csv'):
   
   reclamacoes = []
   #for link in all_hyperlinks:
   for link in all_hyperlinks:
      
      
      try:
         driver.get(link)

         time.sleep(random.randint(5, 8))

         titulo = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-title"]').text
         lugar = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-location"]').text
         data = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-creation-date"]').text
         reclamacao_id = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-id"]').text
         
         reclamacao_id = (reclamacao_id.split('ID: ')[1])

         parsed_data = time.strptime(data, '%d/%m/%Y às %H:%M')

         reclamacao_base = driver.find_element(By.CSS_SELECTOR, '.sc-lzlu7c-17.fRVYjv').text
         interacoes = driver.find_elements(By.CSS_SELECTOR, '[data-testid="complaint-interaction"]')

         interacoes_text = [i.text for i in interacoes]

         again = driver.find_element(By.CSS_SELECTOR, '.sc-uh4o7z-0.bprQlw:nth-child(2) > div').text
         rating = driver.find_element(By.CSS_SELECTOR, '.sc-uh4o7z-0.bprQlw:nth-child(3) > div').text
         status = driver.find_element(By.CSS_SELECTOR, '[data-testid="complaint-status"]').text

         rating = int(rating.split()[0])
         again = again == 'Sim'

         reclamacao = {
            'id': reclamacao_id,
            'titulo': titulo,
            'lugar': lugar,
            'data': parsed_data,
            'reclamacao_base': reclamacao_base,
            'interacoes': interacoes_text,
            'rating': rating,
            'again': again,
            'status': status
         }

         reclamacoes.append(reclamacao)
         
         print(reclamacao)
         
      except:
         
         continue
      
   # transformando em csv as reclamações
   df = pd.DataFrame(reclamacoes)
   df.to_csv(f'reclamacoes/reclamacoes_{empresa}.csv', index=False)
      

else: 
   
   df = pd.read_csv(f'reclamacoes/reclamacoes_{empresa}.csv')
   reclamacoes = df.to_dict(orient='records')
   
   driver.quit()
   
   
print(reclamacoes)



{'id': '206982163', 'titulo': '[Editado pelo Reclame Aqui] ?????', 'lugar': 'Itu - SP', 'data': time.struct_time(tm_year=2025, tm_mon=1, tm_mday=11, tm_hour=12, tm_min=44, tm_sec=0, tm_wday=5, tm_yday=11, tm_isdst=-1), 'reclamacao_base': 'Fechei contrato com a empresa coin nacional a quase 10 meses , paguei cada parcela corretamente e estou a dois meses sem um única resposta da empresa , telefone que está no Google cai em um funerária , msgm não são respondidas e o endereço do CNPJ não consta o escritório no local , por favor se alguém tem algum contato q funcione me ajude !!!', 'interacoes': ['Resposta da empresa\n13/01/2025 às 08:10\nOlá, tudo bem?\n\nAgradecemos por entrar em contato conosco. Lamentamos pelo ocorrido e estamos comprometidos em resolver sua situação da melhor maneira possível.\n\nJá verificamos sua reclamação e nossa equipe entrou em contato para buscar a melhor solução. Caso ainda tenha dúvidas ou precise de mais informações, por favor, fale conosco pelos nossos can